
## Assignment 2 

In [63]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import scipy.stats
from sklearn import metrics

In [158]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

### Cross Validation

In [181]:
# Splitting datasets into training and testing.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [166]:
#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#print(X_train, y_train)

### Likelihood $P(X_i|C_j)$

For individual feature of each category, construct a probabilistic model using Gaussian distribution, where you need to evaluate the mean and standard deviation of each feature across the samples which belong to the category. $X_i$ represents a feature and $C_j$ represents a category. (3/10 marks)

In [192]:
def _pdf(class_idx, x):
        mean = _mean[class_idx]
        var = _var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
def fit(X, y):
        _classes, n_classes, n_features, n_samples = prior(X,y)
        
        # calculate mean, var, and prior for each class
        _mean = np.zeros((n_classes, n_features), dtype=np.float64)
        _var = np.zeros((n_classes, n_features), dtype=np.float64)
        _priors =  np.zeros(n_classes, dtype=np.float64)

        for c in _classes:
            X_c = X[y==c]
            _mean[c, :] = X_c.mean(axis=0)
            _var[c, :] = X_c.var(axis=0)
            _priors[c] = X_c.shape[0] / n_samples
        
        return _classes, _priors, _var, _mean

### Priori $P(C_j)$

Priori probabilities of individual categories can be evaluated using the sample population. (1/10 marks)

In [193]:
def prior(X, y):
    n_samples, n_features = X.shape
    _classes = np.unique(y)
    n_classes = len(_classes)
    
    #print(_classes, n_classes)
    return _classes, n_classes, n_features, n_samples

### Posterior $P(C_j|X)$

Given a testing sample $X$, you can calculate the posterior probability $P(C_j|X)$ of a certain category $C_j$ using Bayes’ theorem and independence assumption of random variables (features). The category with highest posterior probability is voted as the prediction for given testing sample $X$.

In [194]:
def predict(X):
        y_pred = [posterior(x) for x in X]
        return np.array(y_pred)
    
def posterior(x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(_classes):
            prior = np.log(_priors[idx])
            posterior = np.sum(np.log(_pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return _classes[np.argmax(posteriors)]

### Evaluation Confusion Matrix

Calculate the prediction accuracy. (1/10 mark)

In [195]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

_classes, _priors, _var, _mean = fit(X_train, y_train)
#print(_priors)
predictions = predict(X_test)

print("Naive Bayes classification accuracy", str(int(accuracy(y_test, predictions)*100)) + '%')

Naive Bayes classification accuracy 96%
